In [44]:
import subprocess
import json
import numpy as np
from stochopy.optimize import minimize

def run_js_function(lat, long, alt, timeoffset):
    # Command to run the JavaScript script with Node.js
    command = ['node', 'eclipse_explorer_py.jsx', str(lat), str(long), str(alt), str(timeoffset)]

    # Execute the command
    result = subprocess.run(command, capture_output=True, text=True)

    # Check if the command ran successfully
    if result.returncode == 0:
        # Parse the output
        output = json.loads(result.stdout)
        return output
    else:
        # Handle errors if needed
        print("Error:", result.stderr)
        return None

# Example usage
latitude = -86.5
longitude = 10.0
altitude = 250
time_offset = 0.0

output = run_js_function(latitude, longitude, altitude, time_offset)
print(output)

import scipy.optimize as opt
import scipy

# Define your objective function
def calculate_magnitude(x):
    lat, long, alt = x[0],x[1],x[2]
    output = run_js_function(lat, long, alt, 0)
    if output is not None and output['magnitude'] != '':         # - for finding max using fmin
        print(x, -float(output['magnitude']))
        return -float(output['magnitude'])
    else:
        print(x, 0)
        return 0


# LAT: -39, 60)
#TODO: set min magnitude to find to be 1.0 (-1); epsilon: learning rate (want big jumps, not 0.0000..., more like 1)
bounds = [[-86.5, -86.5], [-39.0,60.0],[0,500]]
x = opt.fmin_l_bfgs_b(calculate_magnitude, bounds=bounds, x0=np.array([-86.5, 10.0, 250]) , approx_grad=True, maxiter=100, epsilon=15)
print(x)

# Find the latitude with maximum obscuration for each longitude in the US
def find_max_obscuration_latitudes():
    max_obscuration_latitudes = []
    
    # Iterate over all possible longitudes in the contiguous United States
    for long in range(-160, -10):
        print("Processing longitude:", long)
        # Perform gradient descent to find latitude and altitude with maximum obscuration for the current longitude
        best_lat, best_alt, max_obscuration = gradient_descent_for_longitude(long)
        
        # Append the latitude with maximum obscuration for the current longitude
        max_obscuration_latitudes.append((best_lat, long, best_alt, max_obscuration))
    
    return max_obscuration_latitudes

# Find latitudes with maximum obscuration for each longitude
#max_obscuration_latitudes = find_max_obscuration_latitudes()

# Print the results
#for latitude, longitude, altitude, obscuration in max_obscuration_latitudes:
#    print(f"Latitude with max obscuration for longitude {longitude} at altitude {altitude}: {latitude} (Obscuration: {obscuration})")


# TODO: GOTTA PLAY AROUND WITH ALT VALUES

{'magnitude': ''}
[-86.5  10.  250. ] 0
[-86.5  10.  250. ] 0
[-86.5  25.  250. ] 0
[-86.5  10.  265. ] 0
(array([-86.5,  10. , 250. ]), 0, {'grad': array([nan,  0.,  0.]), 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 4, 'nit': 0, 'warnflag': 2})
